In [ ]:
# Installing the required dependencies
!pip install datasets
!pip install nltk
!pip install bs4
!pip install gensim

In [ ]:
# Loading in the required libraries
from datasets import load_dataset
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import tqdm as tqdm
from bs4 import BeautifulSoup
from gensim.utils import simple_preprocess
from gensim.models.word2vec import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import statistics

In [ ]:
# Importing the IMDb dataset from Hugging Face and creating the training and development splits
imdb_dataset = load_dataset("imdb")
imdb_train_full = imdb_dataset["train"]
imdb_train_dev = imdb_train_full.train_test_split(test_size = 0.2, stratify_by_column = "label", seed = 123)
imdb_train_text = imdb_train_dev["train"]["text"]
imdb_train_y = np.array(imdb_train_dev["train"]["label"])
imdb_dev_text = imdb_train_dev["test"]["text"]
imdb_dev_y = np.array(imdb_train_dev["test"]["label"])

In [ ]:
# Preprocessing our training data (pre-split)
stopwords_list = stopwords.words("english")
processed_reviews_train = []
for review in tqdm.tqdm(imdb_train_full["text"]):
  raw = BeautifulSoup(review)
  html_remove = raw.get_text()
  temp = simple_preprocess(html_remove)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_train.append(temp_update)

# Preprocessing our training data (post-split)
stopwords_list = stopwords.words("english")
processed_reviews_train_post = []
for review in tqdm.tqdm(imdb_train_text):
  raw = BeautifulSoup(review)
  html_remove = raw.get_text()
  temp = simple_preprocess(html_remove)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_train_post.append(temp_update)

# Preprocessing our development data
processed_reviews_dev = []
for review in tqdm.tqdm(imdb_dev_text):
  raw = BeautifulSoup(review)
  html_remove = raw.get_text()
  temp = simple_preprocess(html_remove)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_dev.append(temp_update)

# Defining the final word2vec model parameters
vec_size = 600
window_size = 15
model_architecture = 1
subsample = 1e-2

# Creating the document level representation using the final word2vec model for each review in the training (post-split) and development splits
w2v_model = Word2Vec(sentences = processed_reviews_train, size = vec_size, window = window_size, sg = model_architecture, sample = subsample, seed = 123)

imdb_train_post = np.zeros([len(processed_reviews_train_post), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_train_post))):
  word_list = []
  for word in processed_reviews_train_post[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_train_post[i] = np.mean(w2v_model.wv[word_list], axis = 0)

imdb_dev = np.zeros([len(processed_reviews_dev), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_dev))):
  word_list = []
  for word in processed_reviews_dev[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_dev[i] = np.mean(w2v_model.wv[word_list], axis = 0)      

In [ ]:
# Defining the hyperparameter configurations
penalty_term = ["l1", "l2"]
tolerance = [1e-6, 1e-4, 1e-2]
regularisation = [1/10, 1/8, 1/6, 1/4, 1/2, 1]
max_iterations = [100, 300, 500]
num_models = len(penalty_term)*len(tolerance)*len(regularisation)*len(max_iterations)
models = np.zeros(shape=(num_models, 5))

# Logistic regression hyperparameter tuning
iteration = 0
for p in penalty_term:
  for t in tolerance:
    for r in regularisation:
      for m in max_iterations:
        # Training a Logistic Regression model on the datasets
        imdb_logreg = LogisticRegression(penalty = p, tol = t, C = r, max_iter = m, random_state = 123, solver = 'liblinear')
        imdb_logreg_fit = imdb_logreg.fit(imdb_train_post, imdb_train_y)

        # Using our models to obtain predictions and compute the F1-score
        imdb_logreg_preds = imdb_logreg_fit.predict(imdb_dev)
        imdb_logreg_f1 = f1_score(imdb_dev_y, imdb_logreg_preds)

        # Updating our model matrix
        models[iteration] = [iteration, t, r, m, imdb_logreg_f1]
        print(models[iteration])
        iteration = iteration + 1

In [ ]:
# Defining the hyperparameter configurations
regularisation = [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072]
num_models = len(regularisation)
models = np.zeros(shape=(num_models, 3))

# SVM hyperparameter tuning
iteration = 0
for r in regularisation:
  # Training an SVM model on the datasets
  imdb_svm = SGDClassifier(alpha = r, random_state = 123)
  imdb_svm_fit = imdb_svm.fit(imdb_train_post, imdb_train_y)

  # Using our models to obtain predictions and compute the F1-score
  imdb_svm_preds = imdb_svm_fit.predict(imdb_dev)
  imdb_svm_f1 = f1_score(imdb_dev_y, imdb_svm_preds)

  # Updating our model matrix
  models[iteration] = [iteration, r, imdb_svm_f1]
  print(models[iteration])
  iteration = iteration + 1

In [ ]:
# Defining the hyperparameter configurations
num_trees = [100, 300, 500]
min_samples = [2, 5, 10, 20]
num_features = [1, 5, 17]
num_samples = [0.5, 0.75]
num_models = len(num_trees)*len(min_samples)*len(num_features)*len(num_samples)
models = np.zeros(shape=(num_models, 6))

# Random forests hyperparameter tuning
iteration = 0
for t in num_trees:
  for ns in num_samples:
    for f in num_features:
      for ms in min_samples:
        # Training a Random Forests model on the datasets
        imdb_rf = RandomForestClassifier(n_estimators = t, min_samples_leaf = ms, max_features = f, max_samples = ns, random_state = 123)
        imdb_rf_fit = imdb_rf.fit(imdb_train_post, imdb_train_y)

        # Using our models to obtain predictions and compute the F1-score
        imdb_rf_preds = imdb_rf_fit.predict(imdb_dev)
        imdb_rf_f1 = f1_score(imdb_dev_y, imdb_rf_preds)

        # Updating our model matrix
        models[iteration] = [iteration, t, ns, f, ms, imdb_rf_f1]
        print(models[iteration])
        iteration = iteration + 1